In [ ]:
# to do

\ lire tous les dfs & concat le tt
\ lister cutoff, couper data, train, val, test 
\ 

In [29]:
import pandas as pd
import numpy as np

In [30]:
list_cutoff = ['2017-08-16', '2017-07-31', '2017-07-15', '2017-06-29', '2017-06-13']

In [31]:
path = '/Users/idris/Documents/ds_project/forecast_store_sales_kaggle/data/'

cutoff = '2017-08-16'

In [32]:
df_sales_cutoff = pd.read_csv(path + 'fe/cutoff/{}/sales_cutoff.csv'.format(cutoff), sep=';')
df_enc = pd.read_csv(path + 'fe/cutoff/{}/encoding_agg.csv'.format(cutoff), sep=';').drop(['id', 'bu', 'famid'], axis=1)
df_lag = pd.read_csv(path + 'fe/cutoff/{}/lag_recurisve.csv'.format(cutoff), sep=';').drop('sales', axis=1)
df_rol = pd.read_csv(path + 'fe/cutoff/{}/rol_recurisve.csv'.format(cutoff), sep=';').drop('sales', axis=1)
df_date_hol_oil = pd.read_csv(path + 'fe/date_holidays_oil.csv', sep=';')
df_store_details = pd.read_csv(path + 'fe/stores_details_transactions.csv', sep=';')

df_sales_cutoff['date'] = pd.to_datetime(df_sales_cutoff['date'])
df_enc['date'] = pd.to_datetime(df_enc['date'])
df_lag['date'] = pd.to_datetime(df_lag['date'])
df_rol['date'] = pd.to_datetime(df_rol['date'])
df_date_hol_oil['date'] = pd.to_datetime(df_date_hol_oil['date'])
df_store_details['date'] = pd.to_datetime(df_store_details['date'])

assert df_sales_cutoff.groupby(['date', 'ts_id']).size().max() == 1
assert df_enc.groupby(['date', 'ts_id']).size().max() == 1
assert df_lag.groupby(['date', 'ts_id']).size().max() == 1
assert df_rol.groupby(['date', 'ts_id']).size().max() == 1
assert df_date_hol_oil.groupby(['date']).size().max() == 1
assert df_store_details.groupby(['date', 'bu']).size().max() == 1

In [33]:
df_sales_cutoff.shape, df_enc.shape

((2479012, 7), (2479012, 47))

In [34]:
df_w = pd.merge(df_sales_cutoff, df_enc, how='left', on=['date', 'ts_id'])
assert df_w.shape[0] == df_sales_cutoff.shape[0]
assert df_w.groupby(['date', 'ts_id']).size().max() == 1

In [36]:
df_ww = pd.merge(df_w, df_lag, how='left', on=['date', 'ts_id'])
assert df_ww.shape[0] == df_w.shape[0]
assert df_ww.groupby(['date', 'ts_id']).size().max() == 1

In [37]:
df_www = pd.merge(df_ww, df_rol, how='left', on=['date', 'ts_id'])
assert df_www.shape[0] == df_ww.shape[0]
assert df_www.groupby(['date', 'ts_id']).size().max() == 1

In [38]:
df_wwww = pd.merge(df_www, df_date_hol_oil, how='left', on=['date'])
assert df_wwww.shape[0] == df_www.shape[0]
assert df_wwww.groupby(['date', 'ts_id']).size().max() == 1

In [39]:
df_wwwww = pd.merge(df_wwww, df_store_details, how='left', on=['date', 'bu'])
assert df_wwwww.shape[0] == df_wwww.shape[0]
assert df_wwwww.groupby(['date', 'ts_id']).size().max() == 1

In [40]:
df_wwwww.columns

Index(['ts_id', 'date', 'id', 'bu', 'famid', 'sales', 'onprom',
       'enc_stateid_mean', 'enc_stateid_std', 'enc_stateid_min',
       ...
       'hol_before_2', 'hol_before_3', 'hol_after_1', 'prixoil',
       'transactions', 'isclosed', 'typeid', 'cityid', 'stateid', 'cluster'],
      dtype='object', length=103)

In [41]:
df_wwwww.shape

(2479012, 103)

In [42]:
df_wwwww.to_csv('data/fe/cutoff/{}/final_frame.csv'.format(cutoff), index=False, sep=';')